In [2]:
import numpy as np
import os
import cv2
import pandas
import matplotlib.pyplot as plt
import imutils
from PIL import Image

In [33]:
data_0 = np.load("Data/data0.npy")
data_1 = np.load("Data/data1.npy")
data_2 = np.load("Data/data2.npy")

label_0 = np.load("Label/lab0.npy")
label_1 = np.load("Label/lab1.npy")
label_2 = np.load("Label/lab2.npy")

train_data = np.vstack((data_0, data_1, data_2))
train_lab = np.hstack((label_0, label_1, label_2))
print("Original dataset sizes:", train_data.shape, train_lab.shape)

img_rows = train_data.shape[1]
img_cols = train_data.shape[2]

train_data = train_data.reshape(-1, img_rows, img_cols, 1)
train_data = train_data.reshape(-1, img_rows, img_cols, 1)

print("Reformed dataset sizes:", train_data.shape, train_lab.shape)

Original dataset sizes: (30000, 40, 168) (30000,)
Reformed dataset sizes: (30000, 40, 168, 1) (30000,)


In [ ]:
# for (i,image) in enumerate(train_data):
    
#     im = Image.fromarray(image)
#     im.save("./Converted_images/" +str(i)+".png")

In [138]:
bad_image_count = 0
bad_image_dict = dict()
for (i,image) in enumerate(train_data):
    
    padded_image = cv2.copyMakeBorder(image, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
#     plt.imshow(padded_image)
#     plt.show()
    
    # find the contours in the image
    contours = cv2.findContours(padded_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[1] if imutils.is_cv3() else contours[0]
    
    letter_image_regions = []

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
#         print(x,y,w,h)

        if (w > 40):
            # print("Splitting into 3")
            # This contour is too wide to be a single letter, hence split
            onethird_width = int(w / 3)
            letter_image_regions.append((x, y, onethird_width, h))
            letter_image_regions.append((x + onethird_width, y, onethird_width, h))
            letter_image_regions.append((x + 2*onethird_width, y, onethird_width, h))
        
        elif (w > 20):
            # print("Splitting into 2")
            # This contour is too wide to be a single letter, hence split
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        
        elif(w < 3 or h<9):
            # print("Error")
            # Some error in contouring, ignore!
            continue
        
        else:
            letter_image_regions.append((x, y, w, h))
        
    # If more than 4 contours then skip the image instead of saving bad training data!
    if len(letter_image_regions) != 4:
        
        bad_image_count+=1
        
        if(str(len(letter_image_regions)) not in bad_image_dict):
            bad_image_dict[str(len(letter_image_regions))] = [i]
            
        else:
            bad_image_dict[str(len(letter_image_regions))].append(i)

        continue

    # Sort the detected letter images based on the x coordinate
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])
    
    # Save out each letter as a single image
    for letter_bounding_box in letter_image_regions:
    
        x, y, w, h = letter_bounding_box

        letter_image = padded_image[y-2:y + h+2, x-2:x + w+2]
#         plt.imshow(letter_image)
#         plt.show()

print(bad_image_count)
for key in bad_image_dict:
    print(key,len(bad_image_dict[key]))    
print(bad_image_dict)
        

120
3 48
5 72
{'3': [25, 1283, 2199, 3235, 3965, 4110, 4309, 4762, 4782, 5823, 6211, 6625, 7151, 7214, 8145, 9000, 9644, 9854, 10095, 11353, 11439, 11950, 13874, 14919, 15216, 15598, 16135, 16570, 16957, 17296, 17306, 19091, 19175, 19683, 20045, 20539, 20619, 24026, 24413, 24762, 25360, 25382, 28145, 28304, 28910, 29142, 29454, 29564], '5': [245, 1100, 1848, 1943, 2043, 2733, 3615, 3699, 3744, 4741, 4804, 4828, 5060, 5725, 6134, 7344, 8031, 8906, 9261, 9599, 9971, 10365, 10378, 10699, 11192, 11291, 11772, 11865, 12254, 13332, 13714, 14027, 14832, 14950, 15519, 15720, 16034, 16084, 17180, 17357, 17511, 17582, 18814, 19050, 20959, 21197, 21668, 22141, 22302, 22550, 23017, 23483, 23497, 23707, 24678, 24726, 25283, 25413, 26098, 26137, 26333, 26547, 26788, 27015, 27049, 27087, 27149, 27735, 27927, 28939, 29553, 29819]}
